In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
np.bool = np.bool_
import pymc3 as pm

AttributeError: module 'numpy.distutils.__config__' has no attribute 'blas_opt_info'

In [3]:
def my_f(f,x):
    return f(x)

def g(x):
    return np.exp(x)

my_f(lambda x: x**2, 5)

25

In [4]:
my_f(g, 0)

1.0

In [30]:
import eurostat
df = eurostat.get_data_df('hlth_cd_apr')
df.head()

,freq,mortalit,sex,icd10,unit,geo\TIME_PERIOD,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,A,PRVT,F,A00-A09,NR,AT,7.0,12.0,9.0,6.0,5.0,15.0,10.0,8.0,6.0,13.0
1,A,PRVT,F,A00-A09,NR,BE,25.0,11.0,20.0,26.0,26.0,27.0,45.0,26.0,44.0,39.0
2,A,PRVT,F,A00-A09,NR,BG,5.0,4.0,10.0,11.0,13.0,7.0,4.0,3.0,4.0,5.0
3,A,PRVT,F,A00-A09,NR,CH,14.0,14.0,9.0,9.0,5.0,13.0,13.0,11.0,14.0,10.0
4,A,PRVT,F,A00-A09,NR,CY,6.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,3.0,3.0


In [9]:
df.columns

Index(['freq', 'mortalit', 'sex', 'icd10', 'unit', 'geo\TIME_PERIOD', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')

In [31]:
df.rename(
    {'geo\\TIME_PERIOD' : 'geo'},
    inplace = True,
    axis = 1
)

In [32]:
from country_codes import eurostat_dictionary

In [33]:
df['country'] = df['geo'].replace(eurostat_dictionary)
df.head()

,freq,mortalit,sex,icd10,unit,geo,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,country
0,A,PRVT,F,A00-A09,NR,AT,7.0,12.0,9.0,6.0,5.0,15.0,10.0,8.0,6.0,13.0,Austria
1,A,PRVT,F,A00-A09,NR,BE,25.0,11.0,20.0,26.0,26.0,27.0,45.0,26.0,44.0,39.0,Belgium
2,A,PRVT,F,A00-A09,NR,BG,5.0,4.0,10.0,11.0,13.0,7.0,4.0,3.0,4.0,5.0,Bulgaria
3,A,PRVT,F,A00-A09,NR,CH,14.0,14.0,9.0,9.0,5.0,13.0,13.0,11.0,14.0,10.0,Switzerland
4,A,PRVT,F,A00-A09,NR,CY,6.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,3.0,3.0,Cyprus


In [34]:
EU_countries = ['Belgium', 'Bulgaria', 'Czechia', 'Denmark',
'Germany (until 1990 former territory of the FRG)', 'Germany','Estonia',
'Ireland', 'Greece', 'Spain', 'France', 'Croatia', 'Italy',
'Cyprus', 'Latvia', 'Lithuania', 'Luxembourg', 'Hungary', 'Malta',
'Netherlands', 'Austria', 'Poland', 'Portugal', 'Romania',
'Slovenia', 'Slovakia', 'Finland', 'Sweden', 'United Kingdom',
'Iceland', 'Liechtenstein', 'Norway', 'Switzerland',
'Bosnia and Herzegovina']


In [35]:
df = df[(df.country.isin(EU_countries)) &\
        (df.sex.isin(['F', 'M'])) &\
        (df.mortalit.isin(['PRVT', 'TRT'])) &\
        (df.unit == 'RT') &\
        (df.icd10 == 'TOTAL')]

In [22]:
df.head()

,freq,mortalit,sex,icd10,unit,geo,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,country
6300,A,PRVT,F,TOTAL,RT,AT,95.29,96.16,93.22,93.95,97.52,96.39,92.94,93.14,94.81,99.09,Austria
6301,A,PRVT,F,TOTAL,RT,BE,101.35,99.97,100.34,95.40,99.71,94.13,93.42,92.83,92.39,113.63,Belgium
6302,A,PRVT,F,TOTAL,RT,BG,119.32,126.17,119.29,129.04,128.32,120.14,121.87,119.05,120.05,177.73,Bulgaria
6303,A,PRVT,F,TOTAL,RT,CH,73.75,74.02,73.85,72.70,69.37,68.91,68.45,71.15,67.12,72.59,Switzerland
6304,A,PRVT,F,TOTAL,RT,CY,57.31,49.38,47.75,54.65,55.67,49.98,46.69,50.87,49.03,54.90,Cyprus


In [36]:
df.drop(
    [
        'unit',
        'icd10',
        'geo'
    ],
    axis = 1,
    inplace = True
)

In [37]:
df.head()

,freq,mortalit,sex,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,country
6300,A,PRVT,F,95.29,96.16,93.22,93.95,97.52,96.39,92.94,93.14,94.81,99.09,Austria
6301,A,PRVT,F,101.35,99.97,100.34,95.40,99.71,94.13,93.42,92.83,92.39,113.63,Belgium
6302,A,PRVT,F,119.32,126.17,119.29,129.04,128.32,120.14,121.87,119.05,120.05,177.73,Bulgaria
6303,A,PRVT,F,73.75,74.02,73.85,72.70,69.37,68.91,68.45,71.15,67.12,72.59,Switzerland
6304,A,PRVT,F,57.31,49.38,47.75,54.65,55.67,49.98,46.69,50.87,49.03,54.90,Cyprus


In [38]:
df = df.melt(
    id_vars = ['country', 'sex', 'mortalit'],
    value_vars = ["{}".format(year) for year in np.arange(2011,2021)],
    var_name = "year",
    value_name = "rate"
)

In [39]:
df.head()

,country,sex,mortalit,year,rate
0,Austria,F,PRVT,2011,95.29
1,Belgium,F,PRVT,2011,101.35
2,Bulgaria,F,PRVT,2011,119.32
3,Switzerland,F,PRVT,2011,73.75
4,Cyprus,F,PRVT,2011,57.31
